In [1]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from typing import List
import pandas as pd
import sacrebleu
from rouge_score import rouge_scorer
import evaluate
from rouge_score import rouge_scorer


rouge = evaluate.load('rouge')
sacrebleu = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")


from transformers import AutoTokenizer, AutoModelForCausalLM
from jiwer import wer,mer,wil,visualize_alignment,process_words

# tokenizer = AutoTokenizer.from_pretrained("iocuydi/llama-2-amharic-3784m")

/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# d = pd.read_csv('./amharic_title_generation/gpt4_responses-amharic_title_generation.csv').input.values.tolist()
!ls

Gpt-4                            human-annotation
amharic_mezmur_completion        human-eval.ipynb
amharic_mt_amh-eng               model_evaluation.ipynb
amharic_title_generation         ner&QA_preformance.ipynb
classification_preformance.ipynb predictions
generation_preformance.ipynb


In [5]:
# amharic_llama_our_data
# llama-2-amharic-3784m
#  resources/notebooks/predictions/amharic-llama-all-data.csv

data = pd.read_csv('./GPT-4/gpt4_responses-amharicqa.csv')
print(data.datasource.value_counts())
data.tail()

datasource
amharicqa    299
Name: count, dtype: int64


,instruction,input,ouptput,datasource,response
294,የሚከተለውን ምንባብ ላይ በመመስረት ጥያቄ መልስ,አፈወርቅ ተክሌ የዓለም ሊቀ ሊቃውንት፣ የኪነ ጥበብ ምሑር የተከበሩ ሰዓሊ...,አቶ ተክሌ ማሞ,amharicqa,የተከበሩ አቶ አፈወርቅ ተክሌ አባታቸው አቶ ተክሌ ማሞ ናቸው።
295,የሚከተለውን ምንባብ ላይ በመመስረት ጥያቄ መልስ,አፈወርቅ ተክሌ የዓለም ሊቀ ሊቃውንት፣ የኪነ ጥበብ ምሑር የተከበሩ ሰዓሊ...,በአሥራ አምስት ዓመታቸው,amharicqa,አፈወርቅ ተክሌ ለከፍተኛ ትምህርት በአሥራ አምስት ዓመታቸው ተመረጡ።
296,የሚከተለውን ምንባብ ላይ በመመስረት ጥያቄ መልስ,አፈወርቅ ተክሌ የዓለም ሊቀ ሊቃውንት፣ የኪነ ጥበብ ምሑር የተከበሩ ሰዓሊ...,ከወይዘሮ ፈለቀች የማታወርቅ,amharicqa,የተከበሩ አቶ አፈወርቅ ተክሌ እናታቸው የማታወርቅ ተወለዱ ተባለች።
297,የሚከተለውን ምንባብ ላይ በመመስረት ጥያቄ መልስ,አፈወርቅ ተክሌ የዓለም ሊቀ ሊቃውንት፣ የኪነ ጥበብ ምሑር የተከበሩ ሰዓሊ...,በሎንዶን የኪነ ጥበብ ማእከላዊ ትምሕርት ቤት,amharicqa,የምሕንድስና ትምህርቷን ለማጠናቀቅ አፈወርቅ ተክሌ ለመጀመሪያ ጊዜ የገቡበ...
298,የሚከተለውን ምንባብ ላይ በመመስረት ጥያቄ መልስ,አፈወርቅ ተክሌ የዓለም ሊቀ ሊቃውንት፣ የኪነ ጥበብ ምሑር የተከበሩ ሰዓሊ...,የስሌድ ኪነ ጥበብ ማዕከል,amharicqa,ለንደን ዩኒቨርሲቲ የስሌድ ኪነ ጥበብ ማዕከል ወያያት አፍሪቃዊ ተማሪ ሆነ...


In [6]:
# data=data[data.Datasource=="amharic_spellcheck"]
# # data.drop_duplicates(subset='Input Text', keep="last")
# # data = data.head(1300)
# data = data.fillna('')
# data.tail()

In [7]:
data.iloc[0]

instruction                       የሚከተለውን ምንባብ ላይ በመመስረት ጥያቄ መልስ
input          ንጉሡ ላሊበላ የሚለውን ስም ያገኘው፣ ሲወለድ በንቦች ስለተከበበ ነው። ላ...
ouptput                                                       11
datasource                                             amharicqa
response                           በላሊበላ 11 ውቅር አብያተ ክርስቲያናት አሉ።
Name: 0, dtype: object

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/israela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
def calculate_wer_single(reference, target):
    # Split the reference and target into words
    ref_words = reference.split()
    target_words = target.split()
    
    # Initialize matrix
    d = [[0] * (len(target_words) + 1) for _ in range(len(ref_words) + 1)]
    for i in range(len(ref_words) + 1):
        d[i][0] = i
    for j in range(len(target_words) + 1):
        d[0][j] = j
        
    # Fill matrix
    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(target_words) + 1):
            if ref_words[i-1] == target_words[j-1]:
                cost = 0
            else:
                cost = 1
            d[i][j] = min(d[i-1][j] + 1, d[i][j-1] + 1, d[i-1][j-1] + cost)
    
    return d[len(ref_words)][len(target_words)] / len(ref_words)

def calculate_wer_list(references, targets):
    if len(references) != len(targets):
        raise ValueError("The number of reference and target sentences must be the same.")
    
    total_wer = 0
    for reference, target in zip(references, targets):
        total_wer += calculate_wer_single(reference, target)
    
    # Return the average WER
    average_wer = total_wer / len(references)
    return average_wer


def calculate_bleu_score(references: List[List[str]], candidates: List[str]) -> float:
    scores = []
    for ref, cand in zip(references, candidates):
        candidate_tokens = nltk.word_tokenize(cand)
        reference_tokens = [nltk.word_tokenize(r) for r in ref]
        
        score = sentence_bleu(reference_tokens, candidate_tokens)
        scores.append(score)

    
    print(f" sentence bleu : {sum(scores) / len(scores) if scores else 0}")

    bleu = sacrebleu.compute(predictions=candidates, references=references)
    print(f"SacreBLEU score:{round(bleu['score'],3)}     =   {bleu} ")

    chr = chrf.compute(predictions=candidates, references=references, word_order=2)
    print(f"corpus_chrf score:{round(chr['score'],3)} {chr}")

    results = rouge.compute(predictions=candidates, references=references)
    print(f"Rogue results: {results}")

    vwer = wer(reference=references, hypothesis=candidates)
    vmer = mer(reference=references, hypothesis=candidates)
    vwil = wil(reference=references, hypothesis=candidates)
    cwer = calculate_wer_list(references=references, targets=candidates)
    print(f"cwer {cwer}   wer {vwer}  ,  mer {vmer}  , wil {vwil}")

    out = process_words(reference=references, hypothesis=candidates)
    print(visualize_alignment(out))

    # scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    # scores = scorer.score(candidates,references)
    # print(score)

   





In [11]:
calculate_bleu_score(data.ouptput.values.tolist(), data.response.values.tolist())


/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/israela/anaconda3/envs/orca/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

 sentence bleu : 1.5546547785328543e-232
SacreBLEU score:9.639     =   {'score': 9.63928291714814, 'counts': [673, 385, 228, 151], 'totals': [3654, 3355, 3056, 2758], 'precisions': [18.41817186644773, 11.475409836065573, 7.460732984293194, 5.474981870920957], 'bp': 1.0, 'sys_len': 3654, 'ref_len': 821} 
corpus_chrf score:42.738 {'score': 42.73777876432554, 'char_order': 6, 'word_order': 2, 'beta': 2}
Rogue results: {'rouge1': 0.2822105430801083, 'rouge2': 0.0802675585284281, 'rougeL': 0.2832059245102724, 'rougeLsum': 0.2828077719382067}
cwer 5.557592769549293   wer 4.09325681492109  ,  mer 0.8408488063660478  , wil 0.876661433096018
sentence 1
REF: ***** 11 *** **** ******* ***
HYP: በላሊበላ 11 ውቅር አብያተ ክርስቲያናት አሉ።
         I      I    I       I   I

sentence 2
REF: ***** *** *** *** **** ******* *** ቤተ መድሃኔ ዓለም ***
HYP: ከላሊበላ አስራ አንዱ ውቅር አብያተ ክርስቲያናት ግዙፉ ቤተ መድሃኔ ዓለም ነው።
         I   I   I   I    I       I   I               I

sentence 3
REF: ** ***** ***** *** *** *** **** ******* *** **